In [ ]:
pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade google-api-python-client

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import googleapiclient.discovery
from googleapiclient.discovery import build
import pandas as pd


api_key = 'Replace-Your-API-KEY-HERE' #get your Youtube API key from Google Cloud Platform
channel_ids = ['UCnz-ZXXER4jOvuED5trXfEA',#tech tfq
               'UCNU_lfiiWBdtULKOw6X0Dig',# Krish Naiyak
               'UCYO_jab_esuFRV4b17AJtAw',#3Blue1Brown
               'UCLLw7jmFsvfIVaUFsLs8mlQ', # Luke Barousse 
               'UCiT9RITQ9PW6BhXK0y2jaeg', # Ken Jee
               'UC7cs8q-gJRlGwj4A8OmCmXg', # Alex the analyst
               'UC2UXDak6o7rBm23k3Vv5dww',# Tina Huang
               'UCvjgXvBlbQiydffZU7m1_aw',#The Coding Train
               'UCFp1vaKzpfvoGai0vE5VJ0w',#Guy in a cube
               'UCCezIgC97PvUuR4_gbFUs5g',#Corey Shafer 
               'UCiEmtpFVJjpvdhsQ2QAhxVA' #Guvi
              ]
youtube = build('youtube', 'v3', developerKey=api_key)

In [ ]:
def get_channel_stats(youtube, channel_ids):
        all_data = []
        request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(channel_ids))
        response = request.execute() 
    
        for i in range(len(response['items'])):
            data = dict(Channel_name = response['items'][i]['snippet']['title'],
                    Subscribers = response['items'][i]['statistics']['subscriberCount'],
                    Views = response['items'][i]['statistics']['viewCount'],
                    Total_videos = response['items'][i]['statistics']['videoCount'],
                    playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
        all_data.append(data)
    
        return all_data


In [ ]:
channel_statistics = get_channel_stats(youtube, channel_ids)
channel_data = pd.DataFrame(channel_statistics)
    

In [ ]:
channel_data

,Channel_name,Subscribers,Views,Total_videos,playlist_id
0,Guy in a Cube,343000,27285579,918,UUFp1vaKzpfvoGai0vE5VJ0w


In [ ]:
playlist_id = channel_data.loc[channel_data['Channel_name']=='Guy in a Cube', 'playlist_id'].iloc[0]

In [ ]:
def get_video_ids(youtube, playlist_id):
    
        request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
        response = request.execute()
    
        video_ids = []
    
        for i in range(len(response['items'])):
             video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
             next_page_token = response.get('nextPageToken')
             more_pages = True
     
        while more_pages:
            if next_page_token is None:
                more_pages = False
            else:
                request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
                response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
          
            return video_ids


In [ ]:
 video_ids = get_video_ids(youtube, playlist_id)

In [ ]:
video_ids

['nY54EX6_vd0',
 'pM7LUBfkpDs',
 'eYxRHJ0hB_E',
 'm2lYsQ9t-5k',
 'tMx3rhl43b4',
 'pKkav_W-jiQ',
 '8vGwBedcXQA',
 'AbiYgh9RD2U',
 'N2WLzB8GOlk',
 'ZKodkcOb3WI',
 'WyvfZMaXLBs',
 'uXes0AEQJps',
 'K6DpOJM6b7E',
 'xL6DvHYjBVs',
 'ldTeS-yxpSE',
 'YkNgmVdTD8E',
 'owSaH3DlRrw',
 '7riWWVMMegE',
 'Ks4ekJeKjZY',
 'iaekrjvr3Xk',
 'H6sLCVanIYE',
 'Hpx52kOodPc',
 'k280CpPKJgc',
 'gnTaUbIMfyM',
 'X_1tBbPwBMg',
 'heY92V11knc',
 'twgsMDuwOuo',
 'fh8HpSNNP30',
 'yvV6z6OInEI',
 'zJopprZC-bE',
 '9wLFlaaPJgk',
 'k-4Oud9jkuw',
 'YOWxvT5XLoc',
 '9SprbLQpW8I',
 'leT_dkXwE1c',
 'w_rSlWELBNk',
 'Pdbc_Q7pnD8',
 '6RzC1P_1Elg',
 'KrsNwegpqlk',
 'zI4MV2vNVxY',
 '58rLq8MF2mc',
 '6KG-LogdI9g',
 'XPuGOZdxX5s',
 'cqqyAy5Cemc',
 'wd2c1sjTENQ',
 'WXpxNpnE3D4',
 'ZKvpiOnwAK4',
 'i27vHCVsHp4',
 '3SKSBRFMIoM',
 '-KXDpi_wyD4',
 'ayV1r2vFYnI',
 'vT83hHmX-VM',
 'WQGS48ZcdbE',
 'nMSuWbKUvAY',
 'WgH2Wg_EUNg',
 'GzblE9HjouE',
 '_ggNrxUcCS8',
 'PsuxE6xExOw',
 'a-iA6EG9CzQ',
 'dlZoy8pQWSU',
 '_1QGTmtGcP8',
 'XkjJo7J8CyQ',
 'ey4ZVK

In [ ]:
def get_video_details(youtube, video_ids):
        all_video_stats = []
    
        for i in range(0, len(video_ids), 50):
            request = youtube.videos().list(
                    part='snippet,statistics',
                    id=','.join(video_ids[i:i+50]))
            response = request.execute()
        
        for video in response['items']:
            video_stats = dict(Title = video['snippet']['title'],
                               Published_date = video['snippet']['publishedAt'],
                               Views = video['statistics']['viewCount'],
                               Likes = video['statistics']['likeCount'],
                                                              )
            all_video_stats.append(video_stats)
    
        return all_video_stats

In [ ]:
video_details = get_video_details(youtube, video_ids)
video_data = pd.DataFrame(video_details)

In [ ]:
video_data

,Title,Published_date,Views,Likes
0,Ready to be a Kusto Detective with Azure Data ...,2023-01-31T16:00:04Z,3416,100
1,REPLAY Power BI tips from the Pros - LIVE (Jan...,2023-01-28T16:42:39Z,3325,64
2,What do you want to see from Guy in a Cube? - ...,2023-01-26T16:00:17Z,4614,171
3,What happens when you miss a refresh day with ...,2023-01-25T16:00:35Z,7353,197
4,What the French Toast is a Slowly Changing Dim...,2023-01-24T16:00:26Z,4966,192
5,REPLAY Power BI tips from the Pros - LIVE (Jan...,2023-01-21T16:55:59Z,3229,79
6,Allow users to Show and Hide columns in a Powe...,2023-01-18T16:00:31Z,10904,194
7,What is a Common Table Expression (CTE) and ho...,2023-01-17T16:00:23Z,10530,305
8,REPLAY Starting 2023 w/ Marco Russo LIVE (Jan ...,2023-01-07T16:51:21Z,4579,91
9,"Wait! Using a Power BI Date slicer, what are m...",2023-01-05T16:00:19Z,14152,340


In [ ]:
def get_comments(youtube,video_ids):
        request = youtube.commentThreads().list(
        part="snippet",
        textFormat="textFormatUnspecified",
        videoId='cqTa7cj5ZBw'
    )
        response = request.execute()
        print(response)

In [ ]:
com=get_comments(youtube,video_ids)

{'kind': 'youtube#commentThreadListResponse', 'etag': '6IREQKVh1FOgnab5Rbx6xp_Ghe0', 'nextPageToken': 'QURTSl9pMzhBQ1hoN0Z0bUgxYWVSQm9qV0NGSjAzNnVYUTZ6Ql81NmE2djdhTnQ5WkJ6UjVnUktiUURYQW4zSTZRd2JLSkp4S0FiMUFTSQ==', 'pageInfo': {'totalResults': 20, 'resultsPerPage': 20}, 'items': [{'kind': 'youtube#commentThread', 'etag': 'msX_RIXTZmsI_VP-mWOh0vY8EWI', 'id': 'UgwnJNmcXnuMpyosUM54AaABAg', 'snippet': {'videoId': 'cqTa7cj5ZBw', 'topLevelComment': {'kind': 'youtube#comment', 'etag': 'UN1iQJfeexB62jW-j6UYkBRUqXo', 'id': 'UgwnJNmcXnuMpyosUM54AaABAg', 'snippet': {'videoId': 'cqTa7cj5ZBw', 'textDisplay': 'love it. I have a slideshow of 5 different reports looping by itself right now on a big screen. Is there anyway to make the presentation refresh data without me doing it manually every day?', 'textOriginal': 'love it. I have a slideshow of 5 different reports looping by itself right now on a big screen. Is there anyway to make the presentation refresh data without me doing it manually every day

""
